# Connect Four

# Import the libraries and define global variables (eg the size of the gameboard)

In [1]:
from __future__ import division

import datetime
print(datetime.datetime.now())
import numpy as np
import os
import random
import scipy.misc
import tensorflow as tf
import tensorflow.contrib.slim as slim

global Xdim
global Ydim
global WinNumber  # Number in a row necessary for a win, typically 4
global RewardForWin
global MAX
MAX = 1e308
global game_number_list
global mean_reward_list

# Connect-4 is usually played on a 7x6 gameboard
Xdim = 7
action_space_size=Xdim
Ydim = 6
WinNumber = 4
# Define your incentives
RewardForWin = 100


2022-04-23 18:25:43.061901


C:\Users\randy\anaconda3\envs\amls2\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\randy\anaconda3\envs\amls2\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\randy\anaconda3\envs\amls2\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\randy\anaconda3\envs\amls2\lib\site-packages\tensorflow\python\f

# Set the hyper parameters

In [2]:
batch_size = 64 # How many experiences to use for each training step. Mnih2015=32

update_freq = 8   # Actually, this is frequency inverted, the number of steps between training rounds. Mnih2015=4

targetQ_y = 0.400  # Discount factor on the target Q-values Mnih2015=0.99

pre_train_steps = 500 # How many steps of random actions before training begins
startE = 1 # Starting chance of random action Mnih2015=1
endE = 0.1 # Final chance of random action Mnih2015=0.1
annealing_time = 0.6 # Percent of games to taper off the exploration: 0 = go immediately to endE, 1 = taper exploration during 100% of games, Mnih2015=.02

h_size = 512 # The number of units in the hidden layer. Mnih2015=512

learning_rate = 0.0003 # Mnih2015 = .00025

training_games = 500000 # overnight

model_path = "./Trainings_500Ktraining/models4/dqn_training_reducedLossPenalty" # The path to save our model to.
summary_path = './summaries4/dqn' # The path to save summary statistics to.
load_model = True # Whether to load a saved model.
train_model = False # Whether to train the model
if (train_model==False):
    training_games = 1

PRETRAIN_WINS = False
Pretrain_Scale = 5
PRETRAIN_THREATS = False 
MIRRORED_TRAINING = False
REWARD_FOR_THREAT = False
MINIMAX_TRAINING = False # Not programmed yet
REWARD_EVERY_MOVE = True
TRAIN_MISSED_WINS = False
TRAIN_MISSED_BLOCKS = False
PRIORITIZED_EXPERIENCE = False
TRAIN_AFTER_TESTGAMES = False


# Define sub-routines for displaying the board and checking for a win

In [3]:
#
# DISPLAY THE GAMEBOARD WITH PROPER ORIENTATION
#
def display_gameboard(gameboard):
    ylen = len(gameboard[0])
    xlen = len(gameboard)
    #print(xlen,ylen)
    gameboard_for_display = np.zeros((xlen,ylen),dtype=int)
    for i in range(ylen):
        gameboard_for_display[:,i] = gameboard[:,ylen-1-i]
    print(gameboard_for_display.T)
    #print("\n",gameboard_for_display.T)

In [4]:
#
#  CHECK TO SEE IF THE LATEST MOVE WINS THE GAME
#
def check_for_win_vector(vector):
    reward = 0
    for start in range(0,len(vector)-WinNumber+1):
        win_at_this_startpoint = 1
        for i in range(start, start+WinNumber):
            if (vector[i] != 1):
                win_at_this_startpoint = 0
        if (win_at_this_startpoint):
            reward=1
            return(reward)
    return(reward)
    
def check_for_win(gameboard,x,y):
    horizontalVector = gameboard.T[y]
    #print('horizontalVector=',horizontalVector)
    reward=check_for_win_vector(horizontalVector)
    if reward:
        #print('horizontalVector=',horizontalVector)
        return(reward)
    
    verticalVector = gameboard[x]
    #print('verticalVector= ',verticalVector)
    reward=check_for_win_vector(verticalVector)
    if reward:
        #print('verticalVector= ',verticalVector)
        return(reward)
    
    diagonalVectorUp=np.zeros(max(Xdim,Ydim),dtype=int)
    diagonalVectorDn=np.zeros(max(Xdim,Ydim),dtype=int)
    for i in range(0,max(Xdim,Ydim)):
        ycoord = i
        xcoordUp = x-(y-i) # create vector running down-left to up-right
        if (i < Ydim) and (xcoordUp in range(0,Xdim)):
            diagonalVectorUp[i]= gameboard[xcoordUp][ycoord]
        xcoordDn = x+(y-i) # create vector running up-left to down-right
        if (i < Ydim) and (xcoordDn in range(0,Xdim)):
            diagonalVectorDn[-1-i]= gameboard[xcoordDn][ycoord]
       
    #print('diagonalVectorUp = ',diagonalVectorUp)
    reward=check_for_win_vector(diagonalVectorUp)
    if reward:
        #print('diagonalVectorUp = ',diagonalVectorUp)
        return(reward)
    
    #print('diagonalVectorDn = ',diagonalVectorDn)
    reward=check_for_win_vector(diagonalVectorDn)
    if reward:
        #print('diagonalVectorDn = ',diagonalVectorDn)
        return(reward)
    
    return(0) # reward= 0 if no win

In [5]:
def check_for_possible_win(gameboard):
    possible_win=0
    win_column=0
    for column in range(Xdim):
        gameboard_possible=gameboard.copy()
        if(gameboard[column,-1]==0):
            row = list(gameboard[column]).index(0) # Find the lowest open slot in the chosen column
            gameboard_possible[column,row] = 1
            if check_for_win(gameboard_possible,column,row):
                possible_win+=1
                win_column=column
                #print('\nPossible win in column', win_column)
                #display_gameboard(gameboard_possible)
    return(possible_win,win_column)           

In [6]:
def check_for_possible_win_verbose(gameboard):  #Use this to assist debugging
    possible_win=0
    win_column=0
    for column in range(Xdim):
        gameboard_possible=gameboard.copy()
        if(gameboard[column,-1]==0):
            row = list(gameboard[column]).index(0) # Find the lowest open slot in the chosen column
            gameboard_possible[column,row] = 1
            if check_for_win(gameboard_possible,column,row):
                print('POSSIBLE WIN HERE')
                display_gameboard(gameboard_possible)
                possible_win+=1
                win_column=column
                #print('\nPossible win in column', win_column)
                #display_gameboard(gameboard_possible)
    return(possible_win,win_column)           

# Implement the network

In [7]:
class Qnetwork():
    def __init__(self ,h_size, num_actions, lr, scope):
        with tf.variable_scope(scope):
            # The network recieves a frame from the game, flattened into an array.
            # It then resizes it and processes it through two convolutional layers.
            #self.observation_input =  tf.placeholder(shape=[None, 12, 12, 3],dtype=tf.float32)
            self.observation_input =  tf.placeholder(shape=[None, Xdim, Ydim, 3],dtype=tf.float32) #new
            
            self.conv1 = slim.conv2d(self.observation_input, 64, 
                                     kernel_size=[3,3], stride=[2,2], 
                                     biases_initializer=None,
                                     activation_fn=tf.nn.elu)
            self.conv2 = slim.conv2d(self.conv1, 64, 
                                     kernel_size=[3,3], 
                                     stride=[2,2], 
                                     biases_initializer=None,
                                     activation_fn=tf.nn.elu)

            # We take the output from the final convolutional layer 
            # and split it into separate advantage and value streams.
            self.hidden = slim.fully_connected(slim.flatten(self.conv2), 
                                               h_size, activation_fn=tf.nn.elu)
            self.advantage = slim.fully_connected(self.hidden, num_actions, activation_fn=None,
                                                  biases_initializer=None)
            self.value = slim.fully_connected(self.hidden, 1, activation_fn=None,
                                                  biases_initializer=None)

            # Task 1: Combine advantage and vaule together to get the final Q-values.
            self.q_out = self.value + tf.subtract(self.advantage, 
                                                  tf.reduce_mean(self.advantage,axis=1, keep_dims=True))
            # Task 2: Select the best action given q_out
            self.predict = tf.argmax(self.q_out,1)

            # Below we obtain the loss by taking the sum of squares difference 
            # between the target and prediction Q values.
            self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
            self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
            self.actions_onehot = tf.one_hot(self.actions,num_actions,dtype=tf.float32)

            self.Q = tf.reduce_sum(tf.multiply(self.q_out, self.actions_onehot), axis=1)

            # Task 3: Compute the TD error
            self.td_error = tf.square(self.targetQ - self.Q)
            self.loss = tf.reduce_mean(self.td_error)
            self.trainer = tf.train.AdamOptimizer(learning_rate=lr)
            self.update = self.trainer.minimize(self.loss)

# Experience Replay

In [8]:
class experience_buffer():
    def __init__(self, buffer_size = 50000):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self,experience):
        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience)+len(self.buffer))-self.buffer_size] = []
        self.buffer.extend(experience)
            
    def sample(self,size):
        return np.reshape(np.array(random.sample(self.buffer,size)),[size,5])

# Update the target network

In [9]:
def update_target_graph(from_scope, to_scope):
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)

    op_holder = []
    for from_var,to_var in zip(from_vars,to_vars):
        op_holder.append(to_var.assign(from_var))
    return op_holder

In [10]:
#
# Train the network 
# Definitions
action_space_size = Xdim
tf.reset_default_graph()
mainQN = Qnetwork(h_size, 1, learning_rate, "main")
targetQN = Qnetwork(h_size, 1, learning_rate, "target")

init = tf.global_variables_initializer()

if not os.path.exists(summary_path):
    os.makedirs(summary_path)
    
if not os.path.exists(model_path):
    os.makedirs(model_path)

saver = tf.train.Saver()

trainables = tf.trainable_variables()

update_target_ops = update_target_graph("main", "target")

myBuffer = experience_buffer()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


# LET THE USER PLAY AGAINST THE COMPUTER

In [11]:
def get_user_input(gameboard):
    display_gameboard(gameboard)
    user_string= input('Pick a column to play (1-7)  ')
    while user_string[0] not in ['1','2','3','4','5','6','7']:
        display_game_board(gameboard)
        print('Illegal entry.  Pick a column (1-7)  ')
    column=int(user_string[0])-1
    if gameboard[column,-1]:
        print("That column is already full.  I'll pick for you this time.")
        while gameboard[column,-1]:
            column = random.choice(range(Xdim))
    return(column)


In [12]:
def random_agent_with_winlosssearch_middlestart(gameboard):    
        
    # On the first move, start in the middle
    if (-1) not in gameboard: 
        return(int(Xdim/2))  
    
    # Take a win if you have it
    possible_win,win_column=check_for_possible_win(gameboard)
    if possible_win:
        return(win_column)
    
    # Make a block if you need it
    possible_block,block_column=check_for_possible_win(-1*gameboard)
    if possible_block: # Make a block if you need it
        return(block_column)
    
    Qstate_value = [0.0 for i in range(Xdim)]
    for column in range(Xdim):
        if gameboard[column,-1]:
            Qstate_value[column]= -MAX # don't allow an illegal move into a full column
        else:
            row = list(gameboard[column]).index(0) # Find the lowest open slot in the chosen column #new
            gameboard_possible = gameboard.copy()
            gameboard_possible[column,row] = 1
            possible_loss, loss_column = check_for_possible_win(-1*gameboard_possible)
            if (possible_loss): # avoid giving them an opportunity
                Qstate_value[column] = -MAX/2
            else:
                # Default is random
                Qstate_value[column]=random.random()
                        
    column = Qstate_value.index(max(Qstate_value))
    return(column)


In [13]:
#
# Let the user play a set of games against the semi-random opponent
#

computer_player = random_agent_with_winlosssearch_middlestart
play_yes_no = 'invalid'
done=0
while (play_yes_no[0] != 'n'):
    while (play_yes_no[0] != 'n') and (play_yes_no[0] != 'y'):
        play_yes_no = input('Would you like to play a game against the semi-random opponent? (y/n)')
    if (play_yes_no[0]== 'y'):
        print("\nThe spots you play will be marked 1\nThe spots the computer plays will be marked -1\n")
        play_yes_no = 'invalid' # Loop back after the game to see if we play another game
        gameboard = np.zeros((Xdim,Ydim),dtype=int) 
        while (done==0 and 0 in gameboard):
            # User turn
            user_column = get_user_input(gameboard)
            user_row = list(gameboard[user_column]).index(0)
            gameboard[user_column,user_row]=1
            if check_for_win(gameboard,user_column,user_row):
                display_gameboard(gameboard)
                print('You Win!')
                done=1
            gameboard = -1*gameboard
            #
            # Computer turn
            if (done==0 and 0 in gameboard):
                computer_column = computer_player(gameboard)
                computer_row = list(gameboard[computer_column]).index(0)
                gameboard[computer_column,computer_row]=1
                if check_for_win(gameboard,computer_column,computer_row):
                    display_gameboard(-1*gameboard)
                    print('Too Bad!')
                    done=1
                gameboard = -1*gameboard
        
        if (done==0 and 0 not in gameboard): # Tie game
            display_gameboard(gameboard)
            print('Tie game')
          
print("OK, we'll play later.") # No more games
            
            

Would you like to play a game against the semi-random opponent? (y/n)y

The spots you play will be marked 1
The spots the computer plays will be marked -1

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Pick a column to play (1-7)  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0 -1  0]]
Pick a column to play (1-7)  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0 -1  0  1  0 -1  0]]
Pick a column to play (1-7)  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0 -1  0  1  0 -1  0]]
Pick a column to play (1-7)  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [-1 -1  0  1  0 -1  0]]
Pick a column to play (1-7)  4
[[ 0  

In [14]:
def get_Qvalue(gameboard):
    observations = np.reshape(gameboard, [gameboard.shape[0], gameboard.shape[1], 1])
    observation = np.concatenate([observations, observations, observations], axis=2)
    Qvalue = sess.run(mainQN.q_out,feed_dict={mainQN.observation_input:[observation]})[0]
    return(Qvalue[0])


In [15]:
def trained_agent_minimax_lookahead2_forced(gameboard):

    thresholdA = MAX
    Qstate_valueA = [0.0 for i in range(Xdim)]
    for columnA in range(Xdim):
        gameboardA = gameboard.copy()
        if gameboardA[columnA,-1]:
            Qstate_valueA[columnA]= -MAX # don't allow an illegal move into a full column
        else:
            row = list(gameboardA[columnA]).index(0) # Find the lowest open slot in the chosen column
            gameboardA[columnA,row] = 1 
            if (check_for_win(gameboardA,columnA,row)) or (0 not in gameboardA):
                return(columnA) # Force a winning move
                Qstate_valueA[columnA] = get_Qvalue(gameboardA)
                #if (check_for_win(gameboardA,columnA,row)):
                    #Qstate_valueA[columnA] = MAX/2
                #break # No need to evaluate deeper if this terminates the game
            else:
                 
                thresholdB = MAX
                Qstate_valueB = [0.0 for i in range(Xdim)]
                for columnB in range(Xdim):
                    gameboardB = -1*gameboardA.copy()
                    if gameboardB[columnB,-1]:
                        Qstate_valueB[columnB] = -MAX # don't allow an illegal move into a full column
                    else:
                        row = list(gameboardB[columnB]).index(0) # Find the lowest open slot in the chosen column
                        gameboardB[columnB,row] = 1                        
                        if (check_for_win(gameboardB,columnB,row)) or (0 not in gameboardB):
                            Qstate_valueB[columnB] = get_Qvalue(gameboardB)
                            if (check_for_win(gameboardB,columnB,row)):
                                Qstate_valueB[columnB] = MAX/3  # Prioritize a block
                            #break # No need to evaluate deeper if this terminates the game
                        else:
                        
                            Qstate_valueC = [0.0 for i in range(Xdim)]
                            for columnC in range(Xdim):
                                gameboardC = -1*gameboardB.copy()
                                if gameboardC[columnC,-1]:
                                    Qstate_valueC[columnC] = -MAX # don't allow an illegal move into a full column
                                else:
                                    row = list(gameboardC[columnC]).index(0) # Find the lowest open slot in the chosen column
                                    gameboardC[columnC,row] = 1
                                    if (check_for_win(gameboardC,columnC,row)):
                                        #Qstate_valueC[columnC] = get_Qvalue(gameboardC)
                                        Qstate_valueC[columnC] = MAX/4   # Pioritize a win
                                    else:
                                        Qstate_valueC[columnC]= get_Qvalue(gameboardC)
                                # End of loop of each value of columnC 
                                if (Qstate_valueC[columnC] > thresholdB): # If even one of B's options is higher, we don't need to evaluate further
                                    break #PRUNING
                            # This runs at the end of the columnC loop        
                            #columnCmax = Qstate_valueB.index(max(Qstate_valueC))
                            Qstate_valueC_max = max(Qstate_valueC)
                            if (Qstate_valueC_max<thresholdB):
                                thresholdB = Qstate_valueC_max
                            Qstate_valueB[columnB] = -1*Qstate_valueC_max # Invert the opponent's gain to be our loss
                    
                    # End of loop of each value of columnB
                    if (Qstate_valueB[columnB] > thresholdA): # If even one of B's options is higher, we don't need to evaluate further
                        break #PRUNING       
                
                # This runs at the end of the columnB loop of all values of columnB
                #columnBmax = Qstate_valueB.index(max(Qstate_valueB)) #opponent maximizes their Qvalue for each option
                Qstate_valueB_max = max(Qstate_valueB)
                if (Qstate_valueB_max<thresholdA):
                    thresholdA = Qstate_valueB_max
                Qstate_valueA[columnA] = -1*Qstate_valueB_max # Invert the opponent's gain to be our loss

    count = Qstate_valueA.count(max(Qstate_valueA))
    if count>1:
        columnA = random.randint(0,Xdim-1)
        while Qstate_valueA[columnA] != max(Qstate_valueA):
            columnA=random.randint(0,Xdim-1)
    else:
        columnA = Qstate_valueA.index(max(Qstate_valueA)) # Find the first/only occurence of the max value
    
    return(columnA)


In [16]:
#
# Let the user play against the trained agent
#
playerA=  get_user_input
playerB= trained_agent_minimax_lookahead2_forced
TRAIN_AFTER_TESTGAMES = False


#This is up-to-date altered
#
# Play a set of matches
#
global player1_victories
global player2_victories
player1_victories = 0
player2_victories = 0
playerA_victories = 0
playerB_victories = 0
playerA_possible_wins = 0
playerB_possible_wins = 0
playerA_missed_wins = 0
playerB_missed_wins = 0
playerA_possible_blocks = 0
playerB_possible_blocks = 0
playerA_missed_blocks = 0
playerB_missed_blocks = 0
playerA_created_possible_losses = 0
playerB_created_possible_losses = 0
playerA_starts = 0
playerB_starts = 0
playerA_middle_starts = 0
playerB_middle_starts = 0
playerA_firstplayerwins = 0
playerB_firstplayerwins = 0
tied_games=0
gameboard_history = np.zeros((42,Xdim,Ydim),dtype=int) # use this to use gameboard_history to record all final gameboards
playerA_distribution = [0]*Xdim
playerB_distribution = [0]*Xdim
#gameboard_history = np.zeros((test_games,Xdim,Ydim),dtype=int) # use this to use gameboard_history to record all final gameboards

with tf.Session() as sess:
  sess.run(init)
  ckpt = tf.train.get_checkpoint_state(model_path)
  saver.restore(sess,ckpt.model_checkpoint_path)
  print('Model loaded')
        
  play_yes_no = 'invalid'
  done=0
  game=0
  while (play_yes_no[0] != 'n'):
    while (play_yes_no[0] != 'n') and (play_yes_no[0] != 'y'):
        play_yes_no = input('Would you like to play a game against the trained agent? (y/n)')
    if (play_yes_no[0]=='n'):
        print("OK, we'll play later.") # No more games
        break
    else:    # if (play_yes_no[0]== 'y'):
        print("\nThe spots you play will be marked 1\nThe spots the computer plays will be marked -1\n")
        play_yes_no = 'invalid' # Loop back after the game to see if we play another game
        gameboard = np.zeros((Xdim,Ydim),dtype=int) 
     
        # reset gameboard
        gameboard = np.zeros((Xdim,Ydim),dtype=int) 
        #gameboard_history = np.zeros((Xdim*Ydim,Xdim,Ydim),dtype=int) # Use this to use gameboard_history to record steps in a game
        #observations = np.reshape(gameboard, [gameboard.shape[0], gameboard.shape[1], 1]) # unneeded?
        #observation = np.concatenate([observations, observations, observations], axis=2)  # unneeded?
        done = False
           
        playerA_first = (random.random()<0.5)
        if playerA_first:
            playerA_starts+=1
        else:
            playerB_starts+=1
        episode_steps = 0    
        while not done and 0 in gameboard:
            
            #Player A
            if (episode_steps>0) or playerA_first:
                gameboard *=-1 # switch sides every turn 
                episode_steps+=1
                # Determine our move
                column=playerA(gameboard)
                playerA_distribution[column]+=1 # deleteme
                #display_gameboard(gameboard) # deleteme
                #print('trained', column) # deleteme
                row = list(gameboard[column]).index(0) # Find the lowest open slot in the chosen column 
                gameboard[column,row] = 1 
                # Update our stats
                #done, playerA_victories, playerA_possible_wins, playerA_missed_wins, \
                    #playerA_possible_blocks, playerA_missed_blocks, playerA_created_possible_losses, playerA_middle_starts\
                    #= update_stats(gameboard,column, row,episode_steps, playerA_victories, playerA_possible_wins, playerA_missed_wins, \
                    #playerA_possible_blocks, playerA_missed_blocks, playerA_created_possible_losses, playerA_middle_starts)
                #if done:
                if check_for_win(gameboard,column,row):
                    display_gameboard(gameboard)
                    print("You won!")
                    gameboard_history[game] = gameboard
                    if playerA_first:
                        playerA_firstplayerwins +=1
                    break # PlayerA won!
        
            # Check for a tie
            if (0 not in gameboard):
                tied_games+=1
                gameboard_history[game] = gameboard
                break
            
            #PlayerB        
            gameboard *=-1 # switch sides every turn 
            episode_steps+=1
            # Determine our move
            #display_gameboard(gameboard) # deleteme
            column=playerB(gameboard)
            playerB_distribution[column]+=1 # deleteme
            row = list(gameboard[column]).index(0) # Find the lowest open slot in the chosen column 
            gameboard[column,row] = 1 
            # Update our stats
            #done, playerB_victories, playerB_possible_wins, playerB_missed_wins, \
                #playerB_possible_blocks, playerB_missed_blocks, playerB_created_possible_losses, playerB_middle_starts\
                #= update_stats(gameboard,column, row,episode_steps, playerB_victories, playerB_possible_wins, playerB_missed_wins, \
                #playerB_possible_blocks, playerB_missed_blocks, playerB_created_possible_losses, playerB_middle_starts)
            #if done:
            if (check_for_win(gameboard,column,row)):
                gameboard_history[game] = gameboard
                if not playerA_first:
                    playerB_firstplayerwins +=1
                display_gameboard(-1*gameboard)
                print('Too bad!')  # deleteme
                break # PlayerB won!
            
            # Check for a tie
            if (0 not in gameboard):
                tied_games+=1
                gameboard_history[game] = gameboard
                break
     
        if (TRAIN_AFTER_TESTGAMES):
            # At the end of each game, Get a random batch of experiences.
            trainBatch = myBuffer.sample(batch_size) 
            # Below we perform the Double-DQN update to the target Q-values
            Q1 = sess.run(mainQN.predict, feed_dict={mainQN.observation_input: np.stack(trainBatch[:,3], axis=0)})
            Q2 = sess.run(targetQN.q_out, feed_dict={targetQN.observation_input: np.stack(trainBatch[:,3], axis=0)})
            end_multiplier = -(trainBatch[:,4] - 1)
            doubleQ = Q2[range(batch_size),Q1]
            targetQ = trainBatch[:,2] + (targetQ_y*doubleQ * end_multiplier)
            # Update the network with our target values.
            _, q_loss = sess.run([mainQN.update, mainQN.loss],
                feed_dict={mainQN.observation_input:np.stack(trainBatch[:,0], axis=0),
                        mainQN.targetQ:targetQ, 
                        mainQN.actions:trainBatch[:,1]})
            losses.append(q_loss)


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./Trainings_500Ktraining/models4/dqn_training_reducedLossPenalty\model-499999.cptk
Model loaded
Would you like to play a game against the trained agent? (y/n)n
OK, we'll play later.
